# 계층(layer)에 대한 이해
- 하나의 계층은 하나의 일만 전문적으로 할 수 있어야 한다. (국소적 계산)
- Fully Connected 계층은 WX+B 계산만
- ReLU 계층은 ReLU 연산만
- 덧셈 계층은 덧셈 연산만
- 곱셈 계층은 곱셈 연산만
- 모든 레이어는 미분값을 역전파한다.
  - 순전파 : 계산
  - 역전파 : 미분값을 넘겨 주는 것

In [56]:
# 곱셈 계층 구현하기
# forward : x*y
# backward : dx = 미분값 * y, dy = 미분값 * x

# 비고 : forward할 때 들어온 각 값들은 저장하고 있어야 한다.
# 이제 알고리즘만 알면 forwarding, backwarding 할 수 있음. 레이어를 마음대로 구성 가능

In [57]:
class MulLayer:
  
  # 초기화 : 객체에서 사용할 변수를 미리 준비
  def __init__(self):
    self.x = None
    self.y = None
  
  # 순전파 : x * y 해서 리턴
  # 계산되는 값들 각각 저장하고 있어야 한다.
  def forward(self, x, y):
    self.x = x
    self.y = y

    out = x * y
    return out
  
  # dout : 뒷층에서 넘어온 미분값
  # dx = dout * y
  # dy = dout * x
  
  # return dx, dy
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy    

In [58]:
# 입력값 3개 : 사과의 가격, 사과의 갯수, 소비세
# 레이어 3개
# apple layer, tax layer

apple = 100  # 사과 1개 당 가격
apple_cnt = 2  # 사과 갯수
tax = 1.1  # 소비세

# 계층은 2개
# (apple * apple_cnt) * tax
#       1개          2개

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파 먼저 수행
apple_price = mul_apple_layer.forward(apple, apple_cnt)
price       = mul_tax_layer.forward(apple_price, tax)

print("최종 사과 가격 : {:.0f}".format(price))

최종 사과 가격 : 220


In [59]:
# d돈통 / d포스기
dprice = 1  # 포스기에 찍힌 금액 그대로 돈통에 들어가야 함
dapple_price, dtax = mul_tax_layer.backward(dprice)  # backward 함수 - x, y가 들어갔으니까 dx, dy가 나옴
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)

print("사과 가격 * 사과 개수에 대한 미분값 : {}".format(dapple_price))
print("사과 가격에 대한 미분값 : {}".format(dapple))
print("사과 개수에 대한 미분값 : {}".format(dapple_cnt))
print("소비세에 대한 미분값 : {}".format(dtax))

사과 가격 * 사과 개수에 대한 미분값 : 1.1
사과 가격에 대한 미분값 : 2.2
사과 개수에 대한 미분값 : 110.00000000000001
소비세에 대한 미분값 : 200


In [60]:
# 각각이 미분값의 의미?
# 3. 사과 개수에 대한 미분값?
# 갯수만 바뀌는 거지 나머지 가격, tax는 신경을 안 쓰겠다는 얘기
# 국소적 미분이니까 갯수만 신경씀
# 갯수가 변경되면 가격이 110배 변경된다는 이야기

In [61]:
# 덧셈 계층 구현하기
# forward : x+ y
# backward : 뒷층에서 보내진 미분값에 * 1 을 곱한다
# 비고 : forward 시에 입력된 값을 가지고 있지 않아도 된다. 역전파 시에는 미분값만 필요하니까.

In [62]:
class AddLayer:

  def __init__(self):
    pass
  
  def forward(self, x, y):
    out = x + y
    return out
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [63]:
# 입력값 : 사과의 갯수, 사과의 가격, 귤의 갯수, 귤의 가격, 소비세
apple = 100
apple_cnt = 2

orange = 150
orange_cnt = 3

tax = 1.1

# 1계층
mul_apple_layer = MulLayer()  # 사과 갯수 * 사과 가격
mul_orange_layer = MulLayer()  # 귤 갯수 * 귤 가격

# 2계층
add_apple_orange_layer = AddLayer()  # 사과 총 가격 + 귤 총 가격

# 3계층
mul_tax_layer = MulLayer()  # 과일들의 총 가격 * tax

# 1계층 연산
apple_price = mul_apple_layer.forward(apple, apple_cnt)
orange_price = mul_orange_layer.forward(orange, orange_cnt)

# 2계층 연산
total_fruit_price = add_apple_orange_layer.forward(apple_price, orange_price)

# 3계층 연산
final_money = mul_tax_layer.forward(total_fruit_price, tax)

print("최종 가격 : {}".format(int(final_money)))

최종 가격 : 715


In [64]:
# 국소적 계산이기 때문에
# 3계층 역전파 -> 2계층 역전파를 순서대로 한 다음
# 1계층 역전파를 할 때 사과부터 하나 귤부터 하나는 순간 상관 없음(국소적 계산)

# 역전파
dprice = 1  # d돈통 / d포스기

# dprice / dtotal_price, dprice / dtax
# 이 2개를 구하는 것이 목적
# 세금이 적용되기 전, 세금이 적용될 때의 가격을 구하겠다.
dtotal_price, dtax = mul_tax_layer.backward(dprice)

# d돈통 / d포스기 = d돈통 / dprice * dprice / dtotal_price * dtotal_price / dapple_price
dapple_price, dorange_price = add_apple_orange_layer.backward(dtotal_price)

In [65]:
# 미분값이 곱해진다 -> 미분값이 뒤로 전달된다.

# 미분의 정확한 의미 파악!

In [66]:
dorange, dorange_cnt = mul_orange_layer.backward(dorange_price)
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)

print("사과 2개 오렌지 3개의 가격 : {}".format(price))
print("사과 개수 미분 : {}".format(dapple_cnt))
print("사과 가격 미분 : {}".format(dapple))
print("오렌지 개수 미분 : {}".format(dorange_cnt))
print("오렌지 가격 미분 : {}".format(dorange))
print("소비세 미분 : {}".format(dtax))

사과 2개 오렌지 3개의 가격 : 220.00000000000003
사과 개수 미분 : 110.00000000000001
사과 가격 미분 : 2.2
오렌지 개수 미분 : 165.0
오렌지 가격 미분 : 3.3000000000000003
소비세 미분 : 650


# 신경망 레이어 만들기
- ReLU
- Sigmoid
- Affine 레이어 (기하학 레이어 - Fully Connected, Dense)
- SoftMax + Loss 레이어

## ReLU
- 필요한 부분, 알아내고자 하는 특징들만 강조
- 이미지 분류에 많이 사용

In [67]:
class ReLU:

  # 초기화할 때 가지고 있어야 할 정보?
  # 어떤 위치의 x의 원소가 0보다 작았는 지

  # 그래야지 순전파때 기억하고 있다가 역전파 때 사용
  # 마스킹. 이라고 함 (음수인 데이터의 위치를 찍어놓는 것)
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x <= 0)  # 배열 x에서 어떤 값이 음수인지 마스킹 해놓는다.
                          # 음수인 부분만 True, 나머지 False
                          # (음수부분 체크가 목적)
    out = x.copy()  # 원본 데이터에 손상이 안가게 백업.
    out[self.mask] = 0  # 마스크를 활용해서 음수인 부분만 True  # 조건 index 활용
    
    return out

  # 순전파 때 음수였던 부분을 0으로 만들었음
  # 역전파 때,
  # 음수였었던 index를 기억하고 있다가
  # 미분값 전달시 해당 index을 0으로 만든다.
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx

In [68]:
# 테스트 해보자.
import numpy as np

x = np.array([
              [1.0, -0.5],
              [-2.0, 3.0]
])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [69]:
relu = ReLU()
relu.forward(x)

array([[1., 0.],
       [0., 3.]])

In [70]:
# 마스크 확인
relu.mask

array([[False,  True],
       [ True, False]])

In [71]:
# 음수인 부분만 True 나온 것 확인

In [72]:
dx = np.array([
               [-0.1, 4.0],
               [1.3, -1.1]
])  # 미분된 값
relu.backward(dx)

# 순전파 때 활용된 index 그대로 활용!
# array([[-0.1,  0. ],
#       [ 0. , -1.1]])

array([[-0.1,  0. ],
       [ 0. , -1.1]])

In [73]:
# 이렇게 하는 이유?
# 순전파 때 학습이 안 된건 역전파 때도 신경을 안 쓰겠다는 의미!

# 데이터가 0이라는 것 -> 출력에 영향을 전혀 주지 않는다. -> 학습할 때 전혀 상관이 없는 데이터이다.
# 순전파 때 학습이 안 된 데이터는 미분해봤자 상관이 없다.

## Sigmoid

In [74]:
class Sigmoid:
  def __init__(self):
    self.out = None  # sigmoid의 순전파에서 계산된 값 (y)

  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out
  
    return out
  
  # 시그모이드 함수를 미분하면 dsigmoid = sigmoid*(1-sigmoid)
  def backward(self, dout):
    dx = dout * self.out * (1.0 - self.out)
    return dx

In [75]:
x = np.array([0.991, 0.34, 0.56])
sigmoid = Sigmoid()
print(sigmoid.forward(x))

dout = 1.3
print(sigmoid.backward(dout))

# 지금 우선 값들 마음대로 집어넣음

[0.7292854  0.58419052 0.63645254]
[0.25665667 0.31578554 0.30079492]


## Affine

In [85]:
# Affine 계층의 순전파
X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)

Y = np.dot(X, W) + B
print(Y)

[1.28657099 0.72942172 0.64302392]


In [77]:
X.shape, W.shape, B.shape  # shape가 맞기 때문에 브로드캐스팅이 일어나면서 행렬곱 계산이 된다.

((2,), (2, 3), (3,))

In [90]:
X_dot_W = np.array([
                    [0, 0, 0],  # 1번 데이터
                   [10, 10, 10]])  # 2번 데이터
                   # X와 W의 내적 ***결과***
                   # 곱한 결과가 2행 -> 배치가 2개이다.
B = np.array([1, 2, 3])

print(X_dot_W + B)  # B에서 브로드캐스팅 일어남

[[ 1  2  3]
 [11 12 13]]


In [89]:
X = np.random.rand(5, 2)
W = np.random.rand(2, 3)
np.dot(X, W)  # 이 경우에는 배치 사이즈 = 5

array([[0.32144306, 0.30076966, 0.37508097],
       [0.27851245, 0.23857556, 0.15387697],
       [0.80975635, 0.69580607, 0.46419579],
       [0.37615826, 0.36589747, 0.54716138],
       [0.68695882, 0.55906241, 0.15120225]])

In [91]:
dY = np.array([
               [1, 2, 3],
               [4, 5, 6]
])  # 각 배치에 대한 미분값
dB = np.sum(dY, axis=0)
# 2차원 배열 기준으로 axis=0이라는 것은
# 1차원 배열이 늘어나는 방향
# 밑으로 늘어나니까
# 세로 방향
print(dB)

[5 7 9]


In [100]:
class Affine:

  def __init__(self, W, b):
    # 가중치, 편향 기억해야함
    self.W = W
    self.b = bin
    
    self.x = None
    # 레이어를 클래스화 시키면
    # W, b를 우리가 따로 초기화해서 만들어 넣어줄 것이기 때문에 언제든지 shape 구할 수 있음
    # x는 바뀜.. x의 형상을 모르면 W랑 곱할 방법이 없음
    # forward 시키면서 x를 넣어줌
    # 그럼 backward할 때도 넣어줘야 함
    # 매번 학습할 때마다 넣어줄 수 없으니까
    # 한 번 forwarding 시킬 때 갖고 있게 해줄거임
    
    # 미리 준비를 하겠다.
    self.original_x_shape = None

    # 최적화를 위해서 각 매개변수의 미분값을 가지고 있어야 한다.
    # (가중치 및 편향의 update)
    self.dW = None
    self.db = None
    # 일전의 미분값을 알고 있어야지 갱신이 됨
    # 안 갖고 있으면 갱신이 안 되고 같은 값들만 출력됨
  
  def forward(self, x):  # x : 1개짜리도 들어올 수 있음.
    self.original_x_shape = x.shape
    x = x.reshape(x.shape[0], -1)  
    # 첫 번재 : 배치 크기
    # 우리가 생각하는 모든 레이어의 작동은 배치를 기준
    # 2차원 배열을 기준으로 작동
    
    # 텐서 대응( 배치마다의 데이터 갯수를 따로 평평하게 줄 세웠음)
    # ex) (3, 2, 2) -> 2차원 배열이 3개가 들어있는 3차원 배열을 -> 데이터가 4개가 들어있는 2개의 배치 배열을 만들었다.
    #      데이터 배치 갯수, 데이터 갯수(2, 2)
    # 맨 앞이 배치 갯수
    # 그 뒤 숫자 2개는 데이터 갯수 의미
    # 헷갈리지 말기
    self.x = x
    
    out = np.dot(self.x, self.W) + self.b
    return out  
  
  def backward(self, dout):
    # dW, db 구하는 이유? 나중에 W,b 값을 update 해야하니까
    # dx 구하는 이유? 기울기=변화량=미분값. 입력한거에 대한 변화량을 알려주기 위해
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    # 아직까지는 일자로 쭉 곱할 거기 때문에 돌려줄 필요 없음
    # 돌리는 건 dx를 return 하기 직전에 해주면 됨
    self.db = np.dot(dout, axis=0)
  
    dx = dx.reshape(*self.original_x_shape)
    return dx
    # * : 키워드 argumnet. 튜플을 다 풀어서 넘겨주는
    # * 없으면 튜플 1개만 넘어감
    # * 있으면 튜플 안에 있는 원소들이 제 위치에 알아서 찾아 들어감

In [101]:
# 행렬의 변화 연구하기! 잘 이해하기!

In [102]:
# 출력층을 위한 활성화 계층

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [103]:
class SoftmaxWithLoss:

  def __init__(self):
    self.loss = None  # 확인용
    self.y = None
    self.t = None
    # 원핫인코딩이 되어있는지/안 되어있는지 주의

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)

    return self.loss
  
  def backward(self, dout=1):  
    # 맨마지막 계층의 미분값은 언제나 1 (사실 없어도 되는 값)
    # A - T. 즉, Y - T 만 있으면 된다.
    
    # 배치까지 같이 고려해야하니까 
    # 배치 사이즈 알고 있어야 함
    # (x나 t에 들어있음)
    batch_size = self.t.shape[0]
    # x에서 써도 되는 데 t를 쓴 이유?
    # x는 계속 바뀌기 때문에 불안정해서 t를 사용했음

    # t가 One-hot Encoding 되어있는 경우
    if self.t.size == self.y.size:
      # 이게 참이라면
      # 원핫인코딩이 됬을 때. 라는 의미
      # y의 size는 정해져있음
      # t의 사이즈는 원핫인코딩이 됬을 수도/안 됬을 수도 있음
      # 근데 둘의 사이즈가 같다? -> 원핫인코딩이 되있는 상태이다.
      dx = (self.y - self.t) / batch_size
      # 배치 마다의 오차가 나옴.
      # 데이터 1개당의 오차가 나옴.

    # t가 One-hot Encoding이 안 되어있는 경우
    else:
      dx = self.y.copy()  # 예측값 복사
      # dx : 원핫인코딩이 되어있는 상태
      dx[np.arange(batch_size), self.t] -= 1
      # np.arange(batch_size) : arange에 의해 0~99까지의 배열 만들어짐
      # 정답에 있는 데이터 선택하겠다.
      # y는 지금 형태 softmax
      # 확률에서. 예측을 한 것에서 1을 빼면
      # 오차가 0이라는 이야기

      # 확률에서 1을 빼면 오차가 나온다.
      # 확률 0.8이었는 데 1을 빼면 오차가 0.2가 나온다...?
      # 80% 확률로 예측을 했어. 그러면 오차가 20%
      # 20% 확률로 예측을 했어. 그러면 오차가 80%

      # loss 값 확인하는 것이 중요하지 않음
      
      dx = dx / batch_size
      
    return dx

In [104]:
# 왼쪽 새 폴더 만든 후 common 만든 후
# common.zip 파일 업로드

In [105]:
%cd common
!unzip common.zip

/content/common
Archive:  common.zip
  inflating: functions.py            
  inflating: gradient.py             
  inflating: layers.py               
  inflating: multi_layer_net.py      
  inflating: multi_layer_net_extend.py  
  inflating: optimizer.py            
  inflating: trainer.py              
  inflating: util.py                 
 extracting: __init__.py             


In [106]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)    # 이 부분이 학습 대상 파라미터
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)                                         # 이 부분이 학습 대상 파라미터

        # 계층 생성(매우 중요!!!!!!!!!!!)
        self.layers = OrderedDict()  # 추가되는 데이터의 순서가 유지가 된다.
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])  # 딕셔너리 key, value로 추가
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):  # 계층 하나하나씩 for문 돌아가면서 작동
        for layer in self.layers.values():  # layer 하나씩 꺼냄
            x = layer.forward(x)  # x를 계속 update

        # predict 할 때는 softmax를 활용할 필요 없음. 대소관계 그대로 나오기 때문.
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)  # 이제 LOSS 가 나옴
        # 이 안쪽에서 다 초기화됨
    
    def accuracy(self, x, t):  # predict 한거 보는거
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        # 경사하강법이 아닌 순전파만을 사용한 미분.
        # 수치미분방식.
        
        return grads
        
    def gradient(self, x, t):  # 역전파를 하는 과정
        # 순전파
        self.loss(x, t)  # LOSS 를 우선 구함. 해당 epoch의 오차.
        # 순전파를 통해 LOSS를 구하는 과정

        # 역전파
        dout = 1 # 마지막 계층의 미분값 설정  # LOSS에 대한 LOSS의 미분(돈통에 대한 포스기의 미분 같은 느낌) -> 사실 필요 없음
        dout = self.lastLayer.backward(dout) # 마지막 계층에서의 미분값 전달 받기 (SoftMaxWithLoss에서 받음)
        # 각 softmax 결과마다의 순수한 오차
        
        layers = list(self.layers.values()) # 저장된 레이어를 불러와서 ( 여기서는 순차적인 레이어가 저장 되어 있음 )
        layers.reverse() # 뒤집음(뒤에서 부터 전달해야 하기 때문에 )
        # backward 해야하기 때문에
        
        # 뒤에서 부터 역전파
        for layer in layers:
            dout = layer.backward(dout)  # 오차가 역전파가 되기 시작함. 각 레이어에 들어가기 시작. (미분값을 뒤로 던진다. 전파한다.)
        # numerical_gradient 라는 수치미분 한번도 하지않고 미분값 구할 수 있음

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        # 기울기를 알아냄
        # 이제 기울기를 통해 경사하강법을 함

        return grads

In [ ]:
# 원핫인코딩
# 데이터 플래팅
# normalize 255로 나누는 것
# input_size : 28*28
# hidden_size : 뉴런 갯수
# output_size : 10개

# 수치미분과 오차역전파의 차이